O Business Case visa entender de que forma as features existentes, e as possiveis novas features a serem criadas para responder melhor o modelo, quanto ao valor de frete que vem sendo empregado, no histórico de vendas e entregas.

Considerando o resultado de nossas métricas de medição quanto ao valor de frete, como propriedade intelectual de posse da OLIST:
 1. Propomos a utilização destas informações como insumo para negociação com o mercado de serviços de transporte.
 2. Indicar as os melhores locais de origem para postagem de um produto especifico.

In [2]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
display(dbutils.fs.ls('/mnt/datasets/brazilian-ecommerce/'))

path,name,size
dbfs:/mnt/datasets/brazilian-ecommerce/mkt_datasets_arch.png,mkt_datasets_arch.png,182541
dbfs:/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv,olist_closed_deals_dataset.csv,171352
dbfs:/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv,olist_customers_dataset.csv,9033957
dbfs:/mnt/datasets/brazilian-ecommerce/olist_ecommerce_datasets_arch.png,olist_ecommerce_datasets_arch.png,209677
dbfs:/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv,olist_geolocation_dataset.csv,61273883
dbfs:/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv,olist_marketing_qualified_leads_dataset.csv,703836
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv,olist_order_items_dataset.csv,15438671
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv,olist_order_payments_dataset.csv,5777138
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv,olist_order_reviews_dataset.csv,14409007
dbfs:/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv,olist_orders_dataset.csv,17654914


In [4]:
closed_deals = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv', header = True, sep = ',', inferSchema=True)
customers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv', header = True, sep = ',', inferSchema=True)
geolocation = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv', header = True, sep = ',', inferSchema=True)
marketing = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv', header = True, sep = ',', inferSchema=True)
order_items = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv', header = True, sep = ',', inferSchema=True)
order_payments = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv', header = True, sep = ',', inferSchema=True)
order_review = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv', header = True, sep = ',', inferSchema=True)
orders = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv', header = True, sep = ',', inferSchema=True)
products = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_products_dataset.csv', header = True, sep = ',', inferSchema=True)
sellers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_sellers_dataset.csv', header = True, sep = ',', inferSchema=True)
product_category = spark.read.csv('/mnt/datasets/brazilian-ecommerce/product_category_name_translation.csv', header = True, sep = ',', inferSchema=True)

In [5]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_items.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_items.select(aux))

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,0,0,0,0,0


In [6]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,0,0,160,1783,2965,0


In [7]:
orders = orders.fillna('NA', subset=['order_approved_at'])
orders = orders.fillna('NA', subset=['order_delivered_carrier_date'])
orders = orders.fillna('NA', subset=['order_delivered_customer_date'])

In [8]:
orders = orders.dropna()
orders.count()

Out[432]: 96461

In [9]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,0,0,0,0,0,0


In [10]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_payments.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_payments.select(aux))

order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,0,0,0


In [11]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_review.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_review.select(aux))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1,2330,2497,93013,63321,9164,9187


In [12]:
order_review = order_review.fillna('NA', subset=['review_id'])
order_review = order_review.fillna('NA', subset=['order_id'])
order_review = order_review.fillna('NA', subset=['review_comment_title'])
order_review = order_review.fillna('NA', subset=['review_comment_message'])
order_review = order_review.fillna('NA', subset=['review_creation_date'])
order_review = order_review.fillna('NA', subset=['review_answer_timestamp'])

In [13]:
order_review = order_review.dropna()
order_review.count()

Out[437]: 102692

In [14]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_review.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_review.select(aux))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,0,0,0,0,0


In [15]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,610,610,610,610,2,2,2,2


In [16]:
lenght = products[['product_name_lenght']].agg({'product_name_lenght': 'mean'}).collect()[0][0]
products = products.fillna(lenght, subset=['product_name_lenght'])

descrip = products[['product_description_lenght']].agg({'product_description_lenght': 'mean'}).collect()[0][0]
products = products.fillna(descrip, subset=['product_description_lenght'])

photo = products[['product_photos_qty']].agg({'product_photos_qty': 'mean'}).collect()[0][0]
products = products.fillna(photo, subset=['product_photos_qty'])

weight = products[['product_weight_g']].agg({'product_weight_g': 'mean'}).collect()[0][0]
products = products.fillna(weight, subset=['product_weight_g'])

lenght_cm = products[['product_length_cm']].agg({'product_length_cm': 'mean'}).collect()[0][0]
products = products.fillna(lenght_cm, subset=['product_length_cm'])

height = products[['product_height_cm']].agg({'product_height_cm': 'mean'}).collect()[0][0]
products = products.fillna(height, subset=['product_height_cm'])

width = products[['product_width_cm']].agg({'product_width_cm': 'mean'}).collect()[0][0]
products = products.fillna(width, subset=['product_width_cm'])

In [17]:
products = products.fillna('NA', subset=['product_category_name'])

In [18]:
products = products.dropna()
products.count()

Out[442]: 32951

In [19]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,0,0,0,0,0,0,0


In [20]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in customers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(customers.select(aux))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,0,0,0


In [21]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in sellers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(sellers.select(aux))

seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0,0,0


In [22]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in product_category.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(product_category.select(aux))

product_category_name,product_category_name_english
0,0


In [23]:
#Com estes dados eu crio o codigo do estado para melhorar a acuracia de nosso modelo, podendo utilizar assim o estado como feature na regressao linear
tabelaUF = spark.read.csv('/FileStore/tables/TabelaUF.csv', header = True, sep = ';', inferSchema=True)
tabelaUF = tabelaUF.withColumnRenamed("Código UF", "codigo_UF")
tabelaUF = tabelaUF.withColumnRenamed("Unidade da Federação", "desc_estado")
tabelaUF = tabelaUF.withColumnRenamed("UF", "customer_state")
display(tabelaUF)

codigo_UF,desc_estado,customer_state
11,Rondônia,RO
12,Acre,AC
13,Amazonas,AM
14,Roraima,RR
15,Pará,PA
16,Amapá,AP
17,Tocantins,TO
21,Maranhão,MA
22,Piauí,PI
23,Ceará,CE


In [24]:
#União das bases "orders" com "order_items" utilizando a coluna "order_id".
data_merge = order_items.join(orders, "order_id",how='left')

#União das bases "data_merge" com "order_items" utilizando a coluna "order_id".
data_merge = data_merge.join(order_payments, "order_id",how='left')

#União das bases "data_merge" com "products" utilizando a coluna "product_id".
data_merge = data_merge.join(products, "product_id",how='left')

#União das bases "data_merge" com "customers" utilizando a coluna "customer_id".
data_merge = data_merge.join(customers, "customer_id",how='left')

#União das bases "data_merge" com "Tabelas UF" utilizando a coluna "UF".
data_merge = data_merge.join(tabelaUF, "customer_state" ,how='left')


display(data_merge)

customer_state,customer_id,product_id,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,codigo_UF,desc_estado
RJ,2de342d6e5905a5a8bb3a991c855f3e2,6782d593f63105318f46bbf7633279bf,014405982914c2cde2796ddcf0b8703d,1,325f3178fb58e2a9778334621eecdbf9,2017-08-01T17:50:17.000+0000,27.9,3.81,delivered,2017-07-26T17:38:47.000+0000,2017-07-26T17:50:17.000+0000,2017-07-27T19:39:52.000+0000,2017-07-31T15:53:33.000+0000,2017-08-17T00:00:00.000+0000,1,credit_card,7,78.43,perfumaria,30,487,1,200,16,14,12,e8d87ee946600f7753579a074fbd2d5d,26551,mesquita,33,Rio de Janeiro
RJ,2de342d6e5905a5a8bb3a991c855f3e2,e95ee6822b66ac6058e2e4aff656071a,014405982914c2cde2796ddcf0b8703d,2,a17f621c590ea0fab3d5d883e1630ec6,2017-08-01T17:50:17.000+0000,21.33,25.39,delivered,2017-07-26T17:38:47.000+0000,2017-07-26T17:50:17.000+0000,2017-07-27T19:39:52.000+0000,2017-07-31T15:53:33.000+0000,2017-08-17T00:00:00.000+0000,1,credit_card,7,78.43,esporte_lazer,52,1153,1,180,17,11,14,e8d87ee946600f7753579a074fbd2d5d,26551,mesquita,33,Rio de Janeiro
MS,8cf88d7ba142365ef2ca619ef06f9a0f,e9a69340883a438c3f91739d14d3a56d,019886de8f385a39b75bedbb726fd4ef,1,1b4c3a6f53068f0b6944d2d005c9fc89,2018-02-15T13:08:12.000+0000,159.9,28.5,delivered,2018-02-10T12:52:51.000+0000,2018-02-10T13:08:12.000+0000,2018-02-14T15:28:51.000+0000,2018-02-23T02:03:03.000+0000,2018-03-14T00:00:00.000+0000,1,credit_card,2,188.4,utilidades_domesticas,60,1912,5,3000,33,12,34,d29ede26cd3e2817b314005a88bd28a8,79092,campo grande,50,Mato Grosso do Sul
RS,71accffbcbdf8e02f67a469f65cdbf73,036734b5a58d5d4f46b0616ddc047ced,01a6ad782455876aa89081449d49c452,1,ea8482cd71df3c1969d7b9473ff13abc,2018-01-24T10:17:29.000+0000,34.99,15.1,delivered,2018-01-18T10:07:52.000+0000,2018-01-18T10:17:29.000+0000,2018-01-22T22:37:04.000+0000,2018-02-01T21:02:22.000+0000,2018-02-20T00:00:00.000+0000,1,credit_card,5,50.09,telefonia,58,751,5,300,17,4,12,31c23262d79bc7e803884e37f9bc5359,98801,santo angelo,43,Rio Grande do Sul (*)
SP,d02cc92f5e33eb58d9ff4d5cce6ae901,b1434a8f79cb3528540d9b21e686e823,01d907b3e209269e120a365fc2b97524,1,d1c281d3ae149232351cd8c8cc885f0d,2017-08-16T10:25:08.000+0000,151.99,17.77,delivered,2017-08-09T16:21:06.000+0000,2017-08-10T10:25:08.000+0000,2017-08-11T19:05:53.000+0000,2017-08-16T22:34:11.000+0000,2017-08-29T00:00:00.000+0000,1,credit_card,10,169.76,cama_mesa_banho,57,184,1,13500,55,25,35,a7b781410bcc8bbf3221f48ff45aae6d,8503,ferraz de vasconcelos,35,São Paulo
SP,8c89a09d8fb33b6e5dc8a769d6b2bd63,d86a6c48f83b045cbba6df84926a1f25,028dc52e12ddda803ec1e35eb0b7b0d9,1,c8b3445d737de6befde0c88ede534a5e,2017-12-26T10:53:22.000+0000,49.99,11.73,delivered,2017-12-19T09:43:54.000+0000,2017-12-19T10:53:22.000+0000,2017-12-20T16:49:18.000+0000,2017-12-21T14:22:56.000+0000,2018-01-08T00:00:00.000+0000,1,debit_card,1,61.72,brinquedos,58,1150,2,3900,45,33,26,148a35e3469010c9753897362d1eb8d0,8550,poa,35,São Paulo
RJ,00f5116a953fdf1b86dd0deb055c0e12,aa8d88eb4b9cb38894e33fa624c4287f,036dd381dfb3ec75e0a63e14828cc871,1,6560211a19b47992c3666cc44a7e94c0,2017-09-11T22:43:55.000+0000,55.0,14.14,delivered,2017-09-04T22:25:24.000+0000,2017-09-04T22:43:55.000+0000,2017-09-05T20:49:41.000+0000,2017-09-13T20:36:57.000+0000,2017-09-27T00:00:00.000+0000,1,credit_card,3,69.14,relogios_presentes,54,335,4,250,16,2,11,9f268d26a5ab9839702b621fbcda4f5f,26325,queimados,33,Rio de Janeiro
SP,4024b83c510f004326fbcf0671738663,aa6746e94490239d3d9ee6ab89779aba,03ebfa9712b7dbc7031291856263b314,1,9674754b5a0cb32b638cec001178f799,2018-03-29T18:48:53.000+0000,46.9,8.88,delivered,2018-03-23T11:20:04.000+0000,20

In [25]:
data_merge.dtypes

Out[449]: [('customer_state', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('order_id', 'string'),
 ('order_item_id', 'int'),
 ('seller_id', 'string'),
 ('shipping_limit_date', 'timestamp'),
 ('price', 'double'),
 ('freight_value', 'double'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'timestamp'),
 ('order_approved_at', 'timestamp'),
 ('order_delivered_carrier_date', 'timestamp'),
 ('order_delivered_customer_date', 'timestamp'),
 ('order_estimated_delivery_date', 'timestamp'),
 ('payment_sequential', 'int'),
 ('payment_type', 'string'),
 ('payment_installments', 'int'),
 ('payment_value', 'double'),
 ('product_category_name', 'string'),
 ('product_name_lenght', 'int'),
 ('product_description_lenght', 'int'),
 ('product_photos_qty', 'int'),
 ('product_weight_g', 'int'),
 ('product_length_cm', 'int'),
 ('product_height_cm', 'int'),
 ('product_width_cm', 'int'),
 ('customer_unique_id', 'string'),
 ('customer_zip_code_prefix', 'int'),
 ('customer_city', 'string'),
 ('codigo_UF', 'int'),
 ('desc_estado', 'string')]

In [26]:
data_merge.count()

Out[450]: 117604

In [27]:
display(data_merge.select('price', 
                          'payment_sequential', 
                          'payment_installments', 
                          'payment_value', 
                          'product_name_lenght', 
                          'product_description_lenght',
                          'product_photos_qty', 
                          'product_weight_g', 
                          'product_length_cm', 
                          'product_height_cm', 
                          'product_width_cm', 
                          'customer_zip_code_prefix', 
                          'freight_value')\
        .summary('count',
                'min',
                '25%',
                'mean',
                'stddev',
                '50%',
                '75%',
                '85%',
                '95%',
                '99%',
                'max')
       )

summary,price,payment_sequential,payment_installments,payment_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,freight_value
count,117604,117601,117601,117601,117604,117604,117604,117604,117604,117604,117604,115021,117604
min,0.85,1,0,0.0,5,4,1,0,7,2,6,1003,0.0
25%,39.9,1,1,60.87,43,348,1,300,18,8,15,11310,13.08
mean,120.82284879765531,1.0935281162575148,2.9394818071274904,172.68675189836625,48.75613074385225,786.7269055474303,2.204848474541682,2114.3085694364136,30.25682799904765,16.63018264684875,23.06838202782218,35081.07490806027,20.045551171728174
stddev,184.4773673020208,0.7266915772033292,2.7742227752983504,267.5922896141397,9.962873279822986,648.3003128808833,1.7072406113382372,3788.432456401561,16.189497117017645,13.4576384309911,11.747373446984694,29851.321681181813,15.861350934352723
50%,74.9,1,2,108.22,51,609,2,700,25,13,20,24322,16.29
75%,134.9,1,4,189.24,57,979,3,1800,38,20,30,58801,21.19
85%,180.0,1,6,260.7,59,1278,4,3650,45,27,35,79005,26.64
95%,349.9,1,10,514.7,60,2113,6,9850,62,45,45,90620,45.3
99%,890.0,3,10,1220.64,63,3402,8,18600,95,65,62,97573,85.26


In [28]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in data_merge.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(data_merge.select(aux))

customer_state,customer_id,product_id,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,codigo_UF,desc_estado
2583,2583,0,0,0,0,0,0,0,2583,2583,2583,2583,2583,2583,3,3,3,3,0,0,0,0,0,0,0,0,2583,2583,2583,2583,2583


In [29]:
installments = data_merge[['payment_installments']].agg({'payment_installments': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(lenght, subset=['payment_installments'])

valor_pagamentos = data_merge[['payment_value']].agg({'payment_value': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(lenght, subset=['payment_value'])

lenght = data_merge[['product_name_lenght']].agg({'product_name_lenght': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(lenght, subset=['product_name_lenght'])

descrip = data_merge[['product_description_lenght']].agg({'product_description_lenght': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(descrip, subset=['product_description_lenght'])

photo = data_merge[['product_photos_qty']].agg({'product_photos_qty': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(photo, subset=['product_photos_qty'])

weight = data_merge[['product_weight_g']].agg({'product_weight_g': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(weight, subset=['product_weight_g'])

lenght_cm = data_merge[['product_length_cm']].agg({'product_length_cm': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(lenght_cm, subset=['product_length_cm'])

height = data_merge[['product_height_cm']].agg({'product_height_cm': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(height, subset=['product_height_cm'])

width = data_merge[['product_width_cm']].agg({'product_width_cm': 'mean'}).collect()[0][0]
data_merge = data_merge.fillna(width, subset=['product_width_cm'])

In [30]:
data_merge = data_merge.fillna('NA', subset=['customer_id'])
data_merge = data_merge.fillna('NA', subset=['order_status'])
data_merge = data_merge.fillna('NA', subset=['order_purchase_timestamp'])
data_merge = data_merge.fillna('NA', subset=['order_approved_at'])
data_merge = data_merge.fillna('NA', subset=['order_delivered_carrier_date'])
data_merge = data_merge.fillna('NA', subset=['order_delivered_customer_date'])
data_merge = data_merge.fillna('NA', subset=['order_estimated_delivery_date'])
data_merge = data_merge.fillna('NA', subset=['payment_sequential'])
data_merge = data_merge.fillna('NA', subset=['payment_type'])
data_merge = data_merge.fillna('NA', subset=['product_category_name'])
data_merge = data_merge.fillna('NA', subset=['customer_unique_id'])
data_merge = data_merge.fillna('NA', subset=['customer_zip_code_prefix'])
data_merge = data_merge.fillna('NA', subset=['customer_city'])
data_merge = data_merge.fillna('NA', subset=['customer_state'])

In [31]:
data_merge = data_merge.dropna()
data_merge.count()

Out[455]: 115018

In [32]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in data_merge.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(data_merge.select(aux))

customer_state,customer_id,product_id,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,codigo_UF,desc_estado
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
#Das 117604 linhas presentes após o merge, restaram 115018 linhas após tratamento de valores nulos. Desta forma, foram eliminadas 2586 linhas.
data_merge.count()

Out[457]: 115018

In [34]:
pandas_data_merge = data_merge.toPandas()

In [35]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_data_merge.corr(), xticklabels=pandas_data_merge.corr().columns, yticklabels=pandas_data_merge.corr().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [36]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_data_merge.cov(), xticklabels=pandas_data_merge.cov().columns, yticklabels=pandas_data_merge.cov().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [37]:
'''df = data_merge.select('price', 
                       'payment_sequential', 
                       'payment_installments', 
                       'payment_value', 
                       'product_name_lenght', 
                       'product_description_lenght',
                       'product_photos_qty', 
                       'product_weight_g', 
                       'product_length_cm', 
                       'product_height_cm', 
                       'product_width_cm', 
                       'customer_zip_code_prefix', 
                       'codigo_UF',
                       'freight_value')

display(df)
'''

df = data_merge.select('order_approved_at',
                       'order_delivered_carrier_date',
                       'product_id',
                       'product_category_name',
                       'price', 
                       'payment_value', 
                       'product_photos_qty', 
                       'product_weight_g', 
                       'product_length_cm', 
                       'product_height_cm', 
                       'product_width_cm', 
                       'customer_zip_code_prefix', 
                       'codigo_UF',
                       'freight_value')


from pyspark.sql.functions import unix_timestamp
df = df.withColumn('order_approved_at', unix_timestamp('order_approved_at'))
df = df.withColumn('order_delivered_carrier_date', unix_timestamp('order_delivered_carrier_date'))

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

#para utilização do campo codigo do produto em no modelo de regressao, faço uma tratativa nesta feature pois a mesma esta em formato string, sendo que o modelo exige variaveis numericas
varIdxer = StringIndexer(inputCol='product_id',outputCol='product_id_Indx').fit(df)
df = varIdxer.transform(df)

#para utilização do campo product_category_name em no modelo de regressao, faço uma tratativa nesta feature pois a mesma esta em formato string, sendo que o modelo exige variaveis numericas
varIdxer = StringIndexer(inputCol='product_category_name',outputCol='product_category_name_Indx').fit(df)
df = varIdxer.transform(df)


  
df = df.select('order_approved_at',
                       'order_delivered_carrier_date',
                       'price', 
                       'payment_value', 
                       'product_photos_qty', 
                       'product_weight_g', 
                       'product_length_cm', 
                       'product_height_cm', 
                       'product_width_cm', 
                       'customer_zip_code_prefix', 
                       'codigo_UF',
                       'product_id_Indx',
                       'product_category_name_Indx',
                       'freight_value')  

display(df)

order_approved_at,order_delivered_carrier_date,price,payment_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,codigo_UF,product_id_Indx,product_category_name_Indx,freight_value
1501091417,1501184392,27.9,78.43,1,200,16,14,12,26551,33,3341.0,12.0,3.81
1501091417,1501184392,21.33,78.43,1,180,17,11,14,26551,33,1702.0,2.0,25.39
1518268092,1518622131,159.9,188.4,5,3000,33,12,34,79092,50,151.0,5.0,28.5
1516270649,1516660624,34.99,50.09,5,300,17,4,12,98801,43,199.0,7.0,15.1
1502360708,1502478353,151.99,169.76,1,13500,55,25,35,8503,35,25828.0,0.0,17.77
1513680802,1513788558,49.99,61.72,2,3900,45,33,26,8550,35,3890.0,10.0,11.73
1504565035,1504644581,55.0,69.14,4,250,16,2,11,26325,33,333.0,6.0,14.14
1521830933,1522099078,46.9,55.78,1,500,16,35,25,9280,35,23355.0,25.0,8.88
1533836118,1534164480,59.9,74.63,2,1200,52,13,37,13563,35,7922.0,10.0,14.73
1500980590,1501516175,27.99,101.32,2,500,30,30,30,59151,24,1741.0,5.0,22.67


In [38]:
#excluo as linhas duplicadas
df = df.dropDuplicates()

In [39]:
from pyspark.ml.feature import VectorAssembler

In [40]:
assembler = VectorAssembler(inputCols = df.columns[:-1], outputCol  = 'features')

In [41]:
df_assembler = assembler.transform(df)
display(df_assembler)

order_approved_at,order_delivered_carrier_date,price,payment_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,codigo_UF,product_id_Indx,product_category_name_Indx,freight_value,features
1530807720,1530804360,79.9,103.26,1,1383,50,10,40,38610,31,1.0,0.0,23.36,"List(1, 13, List(), List(1.53080772E9, 1.53080436E9, 79.9, 103.26, 1.0, 1383.0, 50.0, 10.0, 40.0, 38610.0, 31.0, 1.0, 0.0))"
1502124919,1502230679,35.0,102.1,1,300,17,17,11,31930,31,13434.0,16.0,14.1,"List(1, 13, List(), List(1.502124919E9, 1.502230679E9, 35.0, 102.1, 1.0, 300.0, 17.0, 17.0, 11.0, 31930.0, 31.0, 13434.0, 16.0))"
1523807477,1523927782,109.9,133.21,5,1750,30,30,30,25065,33,95.0,9.0,23.31,"List(1, 13, List(), List(1.523807477E9, 1.523927782E9, 109.9, 133.21, 5.0, 1750.0, 30.0, 30.0, 30.0, 25065.0, 33.0, 95.0, 9.0))"
1486596208,1486632029,119.99,103.79,2,800,22,15,27,57072,27,1389.0,11.0,22.34,"List(1, 13, List(), List(1.486596208E9, 1.486632029E9, 119.99, 103.79, 2.0, 800.0, 22.0, 15.0, 27.0, 57072.0, 27.0, 1389.0, 11.0))"
1497949437,1497966522,59.9,76.57,1,7000,26,47,27,24750,33,18.0,5.0,16.67,"List(1, 13, List(), List(1.497949437E9, 1.497966522E9, 59.9, 76.57, 1.0, 7000.0, 26.0, 47.0, 27.0, 24750.0, 33.0, 18.0, 5.0))"
1531355166,1531558980,39.0,58.46,1,700,45,10,35,86310,41,2732.0,0.0,19.46,"List(1, 13, List(), List(1.531355166E9, 1.53155898E9, 39.0, 58.46, 1.0, 700.0, 45.0, 10.0, 35.0, 86310.0, 41.0, 2732.0, 0.0))"
1529492891,1529933460,120.0,167.61,1,4450,104,6,34,3220,35,25404.0,3.0,47.61,"List(1, 13, List(), List(1.529492891E9, 1.52993346E9, 120.0, 167.61, 1.0, 4450.0, 104.0, 6.0, 34.0, 3220.0, 35.0, 25404.0, 3.0))"
1505191827,1505747106,30.0,45.1,4,325,25,25,25,89107,42,5280.0,9.0,15.1,"List(1, 13, List(), List(1.505191827E9, 1.505747106E9, 30.0, 45.1, 4.0, 325.0, 25.0, 25.0, 25.0, 89107.0, 42.0, 5280.0, 9.0))"
1532199850,1532411700,99.99,38.2,1,5950,20,20,20,57608,27,20.0,8.0,88.21,"List(1, 13, List(), List(1.53219985E9, 1.5324117E9, 99.99, 38.2, 1.0, 5950.0, 20.0, 20.0, 20.0, 57608.0, 27.0, 20.0, 8.0))"
1512586286,1513035335,24.99,26.48,6,225,18,4,12,64280,22,3966.0,7.0,34.15,"List(1, 13, List(), List(1.512586286E9, 1.513035335E9, 24.99, 26.48, 6.0, 225.0, 18.0, 4.0, 12.0, 64280.0, 22.0, 3966.0, 7.0))"


In [42]:
(df_assembler.count(), len(df_assembler.columns))

Out[467]: (104041, 15)

In [43]:
treino, teste = df_assembler.randomSplit([0.8, 0.2], seed=36)

In [44]:
from pyspark.ml.regression import LinearRegression

In [45]:
lr = LinearRegression(labelCol= 'freight_value')

In [46]:
modelo = lr.fit(treino)

In [47]:
print(f'Coeficientes: {modelo.coefficients}')
print(f'pValues: {modelo.summary.pValues}')
print(f'Intercepto: {modelo.intercept}')

Coeficientes: [2.825025108285195e-07,-1.7940867729232537e-07,0.007250192649183483,0.011301005558788808,-0.050603348236691874,0.002030322373563793,0.04154186980176931,0.054835117421691865,0.01875751472487894,0.00015734702824595257,-0.5499078300982034,3.2280917515489586e-05,-0.010339966118427009]
pValues: [0.023321829847918085, 0.1504901076112708, 0.0, 0.0, 0.021712656661276863, 0.0, 0.0, 0.0, 5.095146313527366e-06, 0.0, 0.0, 3.850253449400043e-13, 0.004690465646167485, 0.0]
Intercepto: -131.80489677964786

In [48]:
print(f'MAE: {modelo.summary.meanAbsoluteError}') #Erro absoluto do modelo
print(f'MSE: {modelo.summary.meanSquaredError}')
print(f'RMSE: {modelo.summary.rootMeanSquaredError}') #R2 ao quadrado error em valor
print(f'r2 Ajustado: {modelo.summary.r2adj *100}') #As variaveis que nos temos no modelo explicam 53.36% do modelo

MAE: 5.965883586579278
MSE: 119.2042534865159
RMSE: 10.918070044037815
r2 Ajustado: 53.357775630849936

In [49]:
resultado_teste = modelo.evaluate(teste)

In [50]:
print(f'MAE: {resultado_teste.meanAbsoluteError}')
print(f'MSE: {resultado_teste.meanSquaredError}')
print(f'RMSE: {resultado_teste.rootMeanSquaredError}')
print(f'r2 Ajustado: {resultado_teste.r2adj}')

MAE: 5.930230166049064
MSE: 108.83842296677844
RMSE: 10.432565502635411
r2 Ajustado: 0.5195196948808638

In [51]:
display(df_assembler)

order_approved_at,order_delivered_carrier_date,price,payment_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,codigo_UF,product_id_Indx,product_category_name_Indx,freight_value,features
1530807720,1530804360,79.9,103.26,1,1383,50,10,40,38610,31,1.0,0.0,23.36,"List(1, 13, List(), List(1.53080772E9, 1.53080436E9, 79.9, 103.26, 1.0, 1383.0, 50.0, 10.0, 40.0, 38610.0, 31.0, 1.0, 0.0))"
1502124919,1502230679,35.0,102.1,1,300,17,17,11,31930,31,13434.0,16.0,14.1,"List(1, 13, List(), List(1.502124919E9, 1.502230679E9, 35.0, 102.1, 1.0, 300.0, 17.0, 17.0, 11.0, 31930.0, 31.0, 13434.0, 16.0))"
1523807477,1523927782,109.9,133.21,5,1750,30,30,30,25065,33,95.0,9.0,23.31,"List(1, 13, List(), List(1.523807477E9, 1.523927782E9, 109.9, 133.21, 5.0, 1750.0, 30.0, 30.0, 30.0, 25065.0, 33.0, 95.0, 9.0))"
1486596208,1486632029,119.99,103.79,2,800,22,15,27,57072,27,1389.0,11.0,22.34,"List(1, 13, List(), List(1.486596208E9, 1.486632029E9, 119.99, 103.79, 2.0, 800.0, 22.0, 15.0, 27.0, 57072.0, 27.0, 1389.0, 11.0))"
1497949437,1497966522,59.9,76.57,1,7000,26,47,27,24750,33,18.0,5.0,16.67,"List(1, 13, List(), List(1.497949437E9, 1.497966522E9, 59.9, 76.57, 1.0, 7000.0, 26.0, 47.0, 27.0, 24750.0, 33.0, 18.0, 5.0))"
1531355166,1531558980,39.0,58.46,1,700,45,10,35,86310,41,2732.0,0.0,19.46,"List(1, 13, List(), List(1.531355166E9, 1.53155898E9, 39.0, 58.46, 1.0, 700.0, 45.0, 10.0, 35.0, 86310.0, 41.0, 2732.0, 0.0))"
1529492891,1529933460,120.0,167.61,1,4450,104,6,34,3220,35,25404.0,3.0,47.61,"List(1, 13, List(), List(1.529492891E9, 1.52993346E9, 120.0, 167.61, 1.0, 4450.0, 104.0, 6.0, 34.0, 3220.0, 35.0, 25404.0, 3.0))"
1505191827,1505747106,30.0,45.1,4,325,25,25,25,89107,42,5280.0,9.0,15.1,"List(1, 13, List(), List(1.505191827E9, 1.505747106E9, 30.0, 45.1, 4.0, 325.0, 25.0, 25.0, 25.0, 89107.0, 42.0, 5280.0, 9.0))"
1532199850,1532411700,99.99,38.2,1,5950,20,20,20,57608,27,20.0,8.0,88.21,"List(1, 13, List(), List(1.53219985E9, 1.5324117E9, 99.99, 38.2, 1.0, 5950.0, 20.0, 20.0, 20.0, 57608.0, 27.0, 20.0, 8.0))"
1512586286,1513035335,24.99,26.48,6,225,18,4,12,64280,22,3966.0,7.0,34.15,"List(1, 13, List(), List(1.512586286E9, 1.513035335E9, 24.99, 26.48, 6.0, 225.0, 18.0, 4.0, 12.0, 64280.0, 22.0, 3966.0, 7.0))"


In [53]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [54]:
indexer = VectorIndexer(inputCol="features", \
                              outputCol="featureIndexer",\
                              maxCategories=10)


In [55]:
featureIndexer = indexer.fit(df_assembler)

In [56]:
indexedData = featureIndexer.transform(df_assembler)
indexedData.show()

+-----------------+----------------------------+------+-------------+------------------+----------------+-----------------+-----------------+----------------+------------------------+---------+---------------+--------------------------+-------------+--------------------+--------------------+
order_approved_at|order_delivered_carrier_date| price|payment_value|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|customer_zip_code_prefix|codigo_UF|product_id_Indx|product_category_name_Indx|freight_value| features| featureIndexer|
+-----------------+----------------------------+------+-------------+------------------+----------------+-----------------+-----------------+----------------+------------------------+---------+---------------+--------------------------+-------------+--------------------+--------------------+
 1530807720| 1530804360| 79.9| 103.26| 1| 1383| 50| 10| 40| 38610| 31| 1.0| 0.0| 23.36|[1.53080772E9,1.5...|[1.53080772E9,1.5...|
 1502124919| 1502230679| 35.0| 102.1| 1| 300| 17| 17| 11| 31930| 31| 13434.0| 16.0| 14.1|[1.502124919E9,1....|[1.502124919E9,1....|
 1523807477| 1523927782| 109.9| 133.21| 5| 1750| 30| 30| 30| 25065| 33| 95.0| 9.0| 23.31|[1.523807477E9,1....|[1.523807477E9,1....|
 1486596208| 1486632029|119.99| 103.79| 2| 800| 22| 15| 27| 57072| 27| 1389.0| 11.0| 22.34|[1.486596208E9,1....|[1.486596208E9,1....|
 1497949437| 1497966522| 59.9| 76.57| 1| 7000| 26| 47| 27| 24750| 33| 18.0| 5.0| 16.67|[1.497949437E9,1....|[1.497949437E9,1....|
 1531355166| 1531558980| 39.0| 58.46| 1| 700| 45| 10| 35| 86310| 41| 2732.0| 0.0| 19.46|[1.531355166E9,1....|[1.531355166E9,1....|
 1529492891| 1529933460| 120.0| 167.61| 1| 4450| 104| 6| 34| 3220| 35| 25404.0| 3.0| 47.61|[1.529492891E9,1....|[1.529492891E9,1....|
 1505191827| 1505747106| 30.0| 45.1| 4| 325| 25| 25| 25| 89107| 42| 5280.0| 9.0| 15.1|[1.505191827E9,1....|[1.505191827E9,1....|
 1532199850| 1532411700| 99.99| 38.2| 1| 5950| 20| 20| 20| 57608| 27| 20.0| 8.0| 88.21|[1.53219985E9,1.5...|[1.53219985E9,1.5...|
 1512586286| 1513035335| 24.99| 26.48| 6| 225| 18| 4| 12| 64280| 22| 3966.0| 7.0| 34.15|[1.512586286E9,1....|[1.512586286E9,1....|
 1532525415| 1532603760| 9.99| 28.22| 2| 187| 21| 16| 17| 3275| 35| 26091.0| 10.0| 18.23|[1.532525415E9,1....|[1.532525415E9,1....|
 1524948803| 1525102140| 29.9| 48.13| 2| 350| 40| 10| 25| 81220| 41| 4063.0| 6.0| 18.23|[1.524948803E9,1....|[1.524948803E9,1....|
 1496063419| 1496134076|187.57| 219.63| 5| 4755| 43| 40| 31| 9090| 35| 2172.0| 5.0| 32.06|[1.496063419E9,1....|[1.496063419E9,1....|
 1532168122| 1532358420| 118.8| 279.3| 1| 560| 19| 13| 17| 13860| 35| 141.0| 12.0| 2.1|[1.532168122E9,1....|[1.532168122E9,1....|
 1511897748| 1512562174| 21.15| 41.99| 1| 5500| 32| 32| 32| 28930| 33| 1899.0| 5.0| 20.84|[1.511897748E9,1....|[1.511897748E9,1....|
 1485779632| 1485947278| 113.0| 118.05| 1| 1625| 29| 21| 21| 13632| 35| 332.0| 2.0| 18.0|[1.485779632E9,1....|[1.485779632E9,1....|
 1534475727| 1534514280| 29.99| 42.86| 3| 50| 17| 4| 13| 14140| 35| 10008.0| 16.0| 12.87|[1.534475727E9,1....|[1.534475727E9,1....|
 1517698176| 1517936670| 39.0| 58.84| 1| 700| 45| 10| 35| 70234| 53| 2732.0| 0.0| 19.84|[1.517698176E9,1....|[1.517698176E9,1....|
 1522314462| 1522354122| 79.9| 97.05| 2| 100| 16| 16| 11| 13308| 35| 3317.0| 4.0| 17.15|[1.522314462E9,1....|[1.522314462E9,1....|
 1515016766| 1515625698| 37.9| 105.99| 1| 250| 18| 16| 14| 96085| 43| 10405.0| 1.0| 5.03|[1.515016766E9,1....|[1.515016766E9,1....|
+-----------------+----------------------------+------+-------------+------------------+----------------+-----------------+-----------------+----------------+------------------------+---------+---------------+--------------------------+-------------+--------------------+--------------------+
only showing top 20 rows

In [57]:
df_assembler.columns

Out[485]: ['order_approved_at',
 'order_delivered_carrier_date',
 'price',
 'payment_value',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'customer_zip_code_prefix',
 'codigo_UF',
 'product_id_Indx',
 'product_category_name_Indx',
 'freight_value',
 'features']

In [58]:
(trainingData, testData) = df_assembler.randomSplit([0.8, 0.2])

In [59]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

In [60]:
dt = DecisionTreeRegressor(featuresCol="featureIndexer")

In [61]:
model = pipeline.fit(trainingData)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o39301.fit.
: java.lang.IllegalArgumentException: Field "label" does not exist.
Available fields: order_approved_at, order_delivered_carrier_date, price, payment_value, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm, customer_zip_code_prefix, codigo_UF, product_id_Indx, product_category_name_Indx, freight_value, features, featureIndexer
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at scala.collection.MapLike$class.getOrElse(MapLike.scala:128)
	at scala.collection.AbstractMap.getOrElse(Map.scala:59)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:273)
	at org.apache.spark.ml.util.SchemaUtils$.checkNumericType(SchemaUtils.scala:74)
	at org.apache.spark.ml.PredictorParams$class.validateAndTransformSchema(Predictor.scala:55)
	at org.apache.spark.ml.regression.DecisionTreeRegressor.org$apache$spark$ml$tree$DecisionTreeRegressorParams$$super$validateAndTransformSchema(DecisionTreeRegressor.scala:48)
	at org.apache.spark.ml.tree.DecisionTreeRegressorParams$class.validateAndTransformSchema(treeParams.scala:315)
	at org.apache.spark.ml.regression.DecisionTreeRegressor.validateAndTransformSchema(DecisionTreeRegressor.scala:48)
	at org.apache.spark.ml.Predictor.transformSchema(Predictor.scala:146)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:102)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-743078352105704> in <module> 
 ----> 1 model = pipeline . fit ( trainingData ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset

In [62]:
predictions = model.transform(testData)

In [63]:
predictions.select("prediction", "label", "features").show()

In [64]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

treeModel = model.stages[1]

In [65]:
print(treeModel)

REGRESSÃO LINEAR --> Testaremos segmentar o frete por cidade para melhorar o resultado.


ÁRVORE DE REGRESSÃO --> Entender o erro "java.lang.IllegalArgumentException: Field &#34;label&#34; does not exist."